# Importing libraries & dataset

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.preprocessing import StandardScaler, RobustScaler, QuantileTransformer
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import PolynomialFeatures
from sklearn.feature_selection import SelectKBest, f_regression, RFE
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import xgboost as xgb
import lightgbm as lgb
from sklearn.ensemble import VotingRegressor, StackingRegressor
from sklearn.preprocessing import PowerTransformer
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Set random seed for reproducibility
SEED = 42
np.random.seed(SEED)

In [3]:
def rmsle(y_true, y_pred):
    """Calculate Root Mean Squared Logarithmic Error"""
    # Ensure no negative predictions
    y_pred = np.maximum(y_pred, 0)
    y_true = np.maximum(y_true, 0)
    
    # Add small constant to handle zeros
    epsilon = 1e-15
    y_true_log = np.log1p(y_true + epsilon)
    y_pred_log = np.log1p(y_pred + epsilon)
    
    return np.sqrt(mean_squared_error(y_true_log, y_pred_log))

In [4]:
# Load data
train_df = pd.read_csv('../data/train.csv').drop(columns='id')
test_df = pd.read_csv('../data/test.csv')

In [5]:
print(f'Training data shape: {train_df.shape}')
print(f'Test data shape: {test_df.shape}')

Training data shape: (750000, 8)
Test data shape: (250000, 8)


In [6]:
# Remove duplicates
train_df.drop_duplicates(inplace=True)
print(f'Training data shape after removing duplicates: {train_df.shape}')

Training data shape after removing duplicates: (747159, 8)


# Enhanced Data Preprocessing

In [7]:
def advanced_outlier_handling(df, columns, method='isolation_forest'):
    """Advanced outlier detection and handling"""
    from sklearn.ensemble import IsolationForest
    
    df_processed = df.copy()
    
    if method == 'isolation_forest':
        # Use Isolation Forest for multivariate outlier detection
        iso_forest = IsolationForest(contamination=0.05, random_state=SEED)
        outlier_labels = iso_forest.fit_predict(df[columns])
        
        # Keep only inliers (-1 indicates outliers, 1 indicates inliers)
        df_processed = df_processed[outlier_labels == 1]
        print(f"Removed {sum(outlier_labels == -1)} outliers using Isolation Forest")
    
    elif method == 'iqr_clip':
        # Traditional IQR method with clipping
        for col in columns:
            Q1 = df[col].quantile(0.25)
            Q3 = df[col].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR
            df_processed[col] = df_processed[col].clip(lower=lower_bound, upper=upper_bound)
    
    return df_processed

In [8]:
def advanced_feature_engineering(df, has_target=True):
    """Create advanced engineered features"""
    df_eng = df.copy()
    
    df_eng['BMI'] = df_eng['Weight'] / (df_eng['Height'] / 100) ** 2
    df_eng['BMI_Category'] = pd.cut(df_eng['BMI'], 
                                   bins=[0, 18.5, 25, 30, float('inf')], 
                                   labels=['Underweight', 'Normal', 'Overweight', 'Obese'])
    
    
    df_eng['Age_Group'] = pd.cut(df_eng['Age'], 
                                bins=[0, 25, 35, 45, 55, float('inf')], 
                                labels=['Young', 'Adult', 'Middle_Age', 'Senior', 'Elderly'])
    
    
    male_mask = df_eng['Sex'] == 'male'
    female_mask = df_eng['Sex'] == 'female'
    
    df_eng['BMR'] = 0
    df_eng.loc[male_mask, 'BMR'] = (88.362 + 13.397 * df_eng.loc[male_mask, 'Weight'] + 
                                   4.799 * df_eng.loc[male_mask, 'Height'] - 
                                   5.677 * df_eng.loc[male_mask, 'Age'])
    df_eng.loc[female_mask, 'BMR'] = (447.593 + 9.247 * df_eng.loc[female_mask, 'Weight'] + 
                                     3.098 * df_eng.loc[female_mask, 'Height'] - 
                                     4.330 * df_eng.loc[female_mask, 'Age'])
    
    # Heart rate zones
    max_hr = 220 - df_eng['Age']
    df_eng['HR_Reserve'] = max_hr - df_eng['Heart_Rate']
    df_eng['HR_Intensity'] = df_eng['Heart_Rate'] / max_hr
    
    # Exercise intensity categories
    df_eng['Exercise_Intensity'] = pd.cut(df_eng['HR_Intensity'], 
                                         bins=[0, 0.5, 0.7, 0.85, 1.0], 
                                         labels=['Light', 'Moderate', 'Vigorous', 'Maximum'])
    
    # Only create Calories_Per_Minute for training data (which has Calories column)
    if has_target and 'Calories' in df_eng.columns:
        df_eng['Calories_Per_Minute'] = df_eng['Calories'] / df_eng['Duration']
    
    # Weight to height ratio
    df_eng['Weight_Height_Ratio'] = df_eng['Weight'] / df_eng['Height']
    
    # Temperature difference from normal body temp
    df_eng['Temp_Deviation'] = abs(df_eng['Body_Temp'] - 37.0)
    
    # Interaction features
    df_eng['Weight_Duration'] = df_eng['Weight'] * df_eng['Duration']
    df_eng['HR_Duration'] = df_eng['Heart_Rate'] * df_eng['Duration']
    df_eng['BMI_Duration'] = df_eng['BMI'] * df_eng['Duration']
    df_eng['Age_Weight'] = df_eng['Age'] * df_eng['Weight']
    df_eng['BMR_Duration'] = df_eng['BMR'] * df_eng['Duration']
    
    # Polynomial features for key variables
    df_eng['Weight_Squared'] = df_eng['Weight'] ** 2
    df_eng['Duration_Squared'] = df_eng['Duration'] ** 2
    df_eng['HR_Squared'] = df_eng['Heart_Rate'] ** 2
    df_eng['BMI_Squared'] = df_eng['BMI'] ** 2
    
    # Log transformations for skewed features
    df_eng['Log_Weight'] = np.log1p(df_eng['Weight'])
    df_eng['Log_Duration'] = np.log1p(df_eng['Duration'])
    
    return df_eng

In [9]:

def advanced_preprocessing_pipeline(df, target_column=None, is_training=True):
    """Complete preprocessing pipeline"""
    df_processed = df.copy()
    
    # Advanced feature engineering - pass has_target parameter
    df_processed = advanced_feature_engineering(df_processed, has_target=is_training)
    
    # Handle outliers only for training data
    if is_training and target_column and target_column in df_processed.columns:
        numerical_cols = ['Age', 'Height', 'Weight', 'Duration', 'Heart_Rate', 'Body_Temp', 'BMI']
        df_processed = advanced_outlier_handling(df_processed, numerical_cols + [target_column], method='iqr_clip')
    
    # Advanced target transformation for training data
    if is_training and target_column and target_column in df_processed.columns:
        # Apply Box-Cox transformation to target
        from scipy.stats import boxcox
        target_values = df_processed[target_column].values
        
        # Ensure all values are positive for Box-Cox
        if np.any(target_values <= 0):
            target_values = target_values + 1
        
        try:
            transformed_target, lambda_param = boxcox(target_values)
            df_processed[f'{target_column}_boxcox'] = transformed_target
            print(f"Applied Box-Cox transformation with lambda: {lambda_param:.4f}")
        except:
            print("Box-Cox transformation failed, using log transformation")
            df_processed[f'{target_column}_boxcox'] = np.log1p(target_values)
    
    # One-hot encoding for categorical variables
    categorical_cols = ['Sex', 'BMI_Category', 'Age_Group', 'Exercise_Intensity']
    df_processed = pd.get_dummies(df_processed, columns=categorical_cols, drop_first=True)
    
    # Fill any missing values
    numeric_columns = df_processed.select_dtypes(include=[np.number]).columns
    df_processed[numeric_columns] = df_processed[numeric_columns].fillna(df_processed[numeric_columns].median())
    
    return df_processed

In [10]:
# Apply preprocessing
print("Preprocessing training data...")
train_processed = advanced_preprocessing_pipeline(train_df, 'Calories', is_training=True)

print("Preprocessing test data...")
test_processed = advanced_preprocessing_pipeline(test_df, is_training=False)

Preprocessing training data...
Applied Box-Cox transformation with lambda: 0.4398
Preprocessing test data...


In [11]:
# Ensure same columns in train and test (excluding target columns)
target_cols = ['Calories', 'Calories_boxcox', 'Calories_Per_Minute']
train_feature_cols = [col for col in train_processed.columns if col not in target_cols]
test_cols = list(test_processed.columns)

In [12]:
# Add missing columns to test set
for col in train_feature_cols:
    if col not in test_cols:
        test_processed[col] = 0
        print(f"Added missing column to test set: {col}")

# Remove extra columns from test set that aren't in training features
for col in test_cols:
    if col not in train_feature_cols:
        test_processed = test_processed.drop(columns=[col])
        print(f"Removed extra column from test set: {col}")

# Reorder columns to match training features
test_processed = test_processed.reindex(columns=train_feature_cols, fill_value=0)

print(f"Training data shape after preprocessing: {train_processed.shape}")
print(f"Test data shape after preprocessing: {test_processed.shape}")
print(f"Training feature columns: {len(train_feature_cols)}")
print(f"Test feature columns: {len(test_processed.columns)}")

Removed extra column from test set: id
Training data shape after preprocessing: (747159, 37)
Test data shape after preprocessing: (250000, 34)
Training feature columns: 34
Test feature columns: 34


In [13]:
# Prepare features and target
X = train_processed[train_feature_cols]
y = train_processed['Calories']

print(f"Features shape: {X.shape}")
print(f"Target shape: {y.shape}")

Features shape: (747159, 34)
Target shape: (747159,)


In [14]:
scaler = RobustScaler()  # More robust to outliers than StandardScaler
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns, index=X.index)

In [15]:
def create_stratified_split(X, y, test_size=0.2, random_state=SEED):
    """Create stratified split based on target quantiles"""
    y_binned = pd.qcut(y, q=5, labels=False, duplicates='drop')
    return train_test_split(X, y, test_size=test_size, random_state=random_state, stratify=y_binned)

X_train, X_val, y_train, y_val = create_stratified_split(X_scaled, y, test_size=0.2)

In [16]:
def get_advanced_models():
    """Get optimized model configurations"""
    models = {
        'XGBoost': xgb.XGBRegressor(
            n_estimators=1000,
            learning_rate=0.05,
            max_depth=6,
            subsample=0.8,
            colsample_bytree=0.8,
            reg_alpha=0.1,
            reg_lambda=1,
            random_state=SEED,
            early_stopping_rounds=50,
            eval_metric='rmse'
        ),
        'LightGBM': lgb.LGBMRegressor(
            n_estimators=1000,
            learning_rate=0.05,
            max_depth=6,
            subsample=0.8,
            colsample_bytree=0.8,
            reg_alpha=0.1,
            reg_lambda=1,
            random_state=SEED,
            early_stopping_rounds=50,
            metric='rmse'
        ),
        'Random_Forest': RandomForestRegressor(
            n_estimators=300,
            max_depth=15,
            min_samples_split=5,
            min_samples_leaf=2,
            max_features='sqrt',
            random_state=SEED,
            n_jobs=-1
        ),
        'Extra_Trees': ExtraTreesRegressor(
            n_estimators=300,
            max_depth=15,
            min_samples_split=5,
            min_samples_leaf=2,
            max_features='sqrt',
            random_state=SEED,
            n_jobs=-1
        ),
        'Gradient_Boosting': GradientBoostingRegressor(
            n_estimators=500,
            learning_rate=0.05,
            max_depth=6,
            subsample=0.8,
            random_state=SEED
        )
    }
    return models

In [17]:
# Cross-validation with RMSLE
def cross_validate_rmsle(model, X, y, cv=5):
    """Perform cross-validation with RMSLE scoring"""
    kfold = KFold(n_splits=cv, shuffle=True, random_state=SEED)
    rmsle_scores = []
    
    for train_idx, val_idx in kfold.split(X):
        X_train_cv, X_val_cv = X.iloc[train_idx], X.iloc[val_idx]
        y_train_cv, y_val_cv = y.iloc[train_idx], y.iloc[val_idx]
        
        # Fit model
        if hasattr(model, 'early_stopping_rounds'):
            model.fit(X_train_cv, y_train_cv, 
                     eval_set=[(X_val_cv, y_val_cv)])
            
        else:
            model.fit(X_train_cv, y_train_cv)
        
        # Predict and calculate RMSLE
        y_pred = model.predict(X_val_cv)
        rmsle_score = rmsle(y_val_cv, y_pred)
        rmsle_scores.append(rmsle_score)
    
    return np.mean(rmsle_scores), np.std(rmsle_scores)

In [18]:
# Train and evaluate models
models = get_advanced_models()
model_results = {}

print("Training and evaluating models...")
for name, model in models.items():
    print(f"\nTraining {name}...")
    
    # Cross-validation
    cv_mean, cv_std = cross_validate_rmsle(model, X_train, y_train, cv=5)
    
    # Train on full training set
    if hasattr(model, 'early_stopping_rounds'):
        model.fit(X_train, y_train, 
                 eval_set=[(X_val, y_val)])
    else:
        model.fit(X_train, y_train)
    
    # Validation predictions
    y_pred = model.predict(X_val)
    val_rmsle = rmsle(y_val, y_pred)
    
    model_results[name] = {
        'model': model,
        'cv_mean': cv_mean,
        'cv_std': cv_std,
        'val_rmsle': val_rmsle
    }
    
    print(f"CV RMSLE: {cv_mean:.5f} (+/- {cv_std:.5f})")
    print(f"Validation RMSLE: {val_rmsle:.5f}")

Training and evaluating models...

Training XGBoost...
[0]	validation_0-rmse:59.36107
[1]	validation_0-rmse:56.44386
[2]	validation_0-rmse:53.67394
[3]	validation_0-rmse:51.05760
[4]	validation_0-rmse:48.55221
[5]	validation_0-rmse:46.17618
[6]	validation_0-rmse:43.93320
[7]	validation_0-rmse:41.78926
[8]	validation_0-rmse:39.75017
[9]	validation_0-rmse:37.83283
[10]	validation_0-rmse:35.99422
[11]	validation_0-rmse:34.26038
[12]	validation_0-rmse:32.60376
[13]	validation_0-rmse:31.03101
[14]	validation_0-rmse:29.53682
[15]	validation_0-rmse:28.11950
[16]	validation_0-rmse:26.77038
[17]	validation_0-rmse:25.50196
[18]	validation_0-rmse:24.29029
[19]	validation_0-rmse:23.13657
[20]	validation_0-rmse:22.04100
[21]	validation_0-rmse:21.00102
[22]	validation_0-rmse:20.01524
[23]	validation_0-rmse:19.08382
[24]	validation_0-rmse:18.20243
[25]	validation_0-rmse:17.37126
[26]	validation_0-rmse:16.57359
[27]	validation_0-rmse:15.82678
[28]	validation_0-rmse:15.11017
[29]	validation_0-rmse:14.4

In [19]:
# Create advanced ensemble
def create_weighted_ensemble(models_dict, X_train, y_train, X_val, y_val):
    """Create weighted ensemble based on validation performance"""
    
    # Get validation predictions for each model
    val_predictions = {}
    weights = {}
    
    total_inverse_error = 0
    for name, model_info in models_dict.items():
        model = model_info['model']
        val_pred = model.predict(X_val)
        val_predictions[name] = val_pred
        
        # Weight inversely proportional to validation error
        error = model_info['val_rmsle']
        weight = 1 / (error + 1e-8)
        weights[name] = weight
        total_inverse_error += weight
    
    # Normalize weights
    for name in weights:
        weights[name] /= total_inverse_error
    
    print("Ensemble weights:")
    for name, weight in weights.items():
        print(f"{name}: {weight:.4f}")
    
    # Create weighted ensemble predictions
    ensemble_val_pred = np.zeros(len(X_val))
    for name, pred in val_predictions.items():
        ensemble_val_pred += weights[name] * pred
    
    ensemble_rmsle = rmsle(y_val, ensemble_val_pred)
    print(f"\nEnsemble Validation RMSLE: {ensemble_rmsle:.5f}")
    
    return weights, ensemble_rmsle

In [20]:
# Create ensemble
ensemble_weights, ensemble_score = create_weighted_ensemble(model_results, X_train, y_train, X_val, y_val)

Ensemble weights:
XGBoost: 0.2048
LightGBM: 0.2070
Random_Forest: 0.2072
Extra_Trees: 0.1769
Gradient_Boosting: 0.2040

Ensemble Validation RMSLE: 0.06029


In [ ]:
# Retrain on full dataset
print("\nRetraining models on full dataset...")
X_full = pd.concat([X_train, X_val])
y_full = pd.concat([y_train, y_val])

final_models = {}
for name, model_info in model_results.items():
    model = model_info['model']
    
    if name == 'XGBoost':
        params = model.get_params()
        # Remove early stopping for full dataset training
        params['early_stopping_rounds'] = None
        fresh_model = xgb.XGBRegressor(**params)

    elif name == 'LightGBM':
        params = model.get_params()
        # Remove early stopping for full dataset training
        params['early_stopping_rounds'] = None
        fresh_model = lgb.LGBMRegressor(**params)

    else:
        fresh_model = model.__class__(**model.get_params())
    
    fresh_model.fit(X_full, y_full)
    final_models[name] = fresh_model
    print(f"Retrained {name} on full dataset")


Retraining models on full dataset...
Retrained XGBoost on full dataset
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.114299 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3351
[LightGBM] [Info] Number of data points in the train set: 747159, number of used features: 34
[LightGBM] [Info] Start training from score 88.362592
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, 

In [23]:
# Generate predictions
print("\nGenerating test predictions...")
test_scaled = pd.DataFrame(scaler.transform(test_processed), 
                          columns=test_processed.columns, 
                          index=test_processed.index)

print(f"Test data shape for prediction: {test_scaled.shape}")
print(f"Training features shape: {X_scaled.shape}")


Generating test predictions...
Test data shape for prediction: (250000, 34)
Training features shape: (747159, 34)


In [24]:
# Verify column alignment
if list(test_scaled.columns) != list(X_scaled.columns):
    print("WARNING: Column mismatch between train and test!")
    print(f"Train columns: {len(X_scaled.columns)}")
    print(f"Test columns: {len(test_scaled.columns)}")
    # Ensure test has same columns as train in same order
    test_scaled = test_scaled.reindex(columns=X_scaled.columns, fill_value=0)

In [25]:
# Individual model predictions
test_predictions = {}
for name, model in final_models.items():
    pred = model.predict(test_scaled)
    test_predictions[name] = pred

In [26]:
# Ensemble prediction
ensemble_pred = np.zeros(len(test_scaled))
for name, pred in test_predictions.items():
    ensemble_pred += ensemble_weights[name] * pred

# Ensure non-negative predictions
ensemble_pred = np.maximum(ensemble_pred, 0)

In [27]:
# Create submission
submission = pd.DataFrame({
    'id': test_df['id'],
    'Calories': ensemble_pred
})

# Save submission
submission.to_csv('submission_improved.csv', index=False)
print(f"\nSubmission saved! Shape: {submission.shape}")
print(f"Prediction range: {ensemble_pred.min():.2f} to {ensemble_pred.max():.2f}")
print(f"Mean prediction: {ensemble_pred.mean():.2f}")


Submission saved! Shape: (250000, 2)
Prediction range: 1.03 to 290.59
Mean prediction: 88.24


In [28]:
# Feature importance analysis
def analyze_feature_importance(models_dict):
    """Analyze feature importance across models"""
    importance_data = {}
    
    for name, model in models_dict.items():
        if hasattr(model, 'feature_importances_'):
            importance_data[name] = model.feature_importances_
    
    if importance_data:
        importance_df = pd.DataFrame(importance_data, index=X.columns)
        importance_df['Average'] = importance_df.mean(axis=1)
        importance_df = importance_df.sort_values('Average', ascending=False)
        
        print("\nTop 20 most important features:")
        print(importance_df['Average'].head(20))
        
        return importance_df
    
    return None

feature_importance = analyze_feature_importance(final_models)


Top 20 most important features:
HR_Duration            762.319614
Heart_Rate             619.222325
HR_Intensity           599.021076
Age_Weight             362.402814
HR_Reserve             350.205904
BMI_Duration           311.628954
BMR                    292.200621
BMR_Duration           272.221913
Duration               264.470960
Weight_Duration        244.821650
Age                    228.801584
Weight                 222.400390
BMI                    201.000098
Body_Temp              193.020137
Sex_male               187.602701
Weight_Height_Ratio    184.000277
Height                 163.600157
HR_Squared             125.223062
Duration_Squared        60.955942
Temp_Deviation          45.415912
Name: Average, dtype: float64


In [29]:
print("\n" + "="*50)
print("SUMMARY")
print("="*50)
print(f"Best single model: {min(model_results.items(), key=lambda x: x[1]['val_rmsle'])[0]}")
print(f"Best single model RMSLE: {min(model_results.values(), key=lambda x: x['val_rmsle'])['val_rmsle']:.5f}")
print(f"Ensemble RMSLE: {ensemble_score:.5f}")
print("Submission file: submission_improved.csv")


SUMMARY
Best single model: Random_Forest
Best single model RMSLE: 0.06084
Ensemble RMSLE: 0.06029
Submission file: submission_improved.csv
